## Notebook to use ibm-watsonx-gov-catalog-mcp-server in a langchain application 


### Install the dependencies
Note : Restart the kernel after the pip install

In [ ]:
!pip install 'ibm_watsonx_gov_catalog_mcp_server-0.1.0-py3-none-any.whl'
!pip install uv langchain-mcp-adapters langgraph langchain  langchain_openai rich

### Set the needed environment variable 

The environment variables need to be set are:

For OpenAI model:
1. **OPENAI_API_KEY:** This is required for OpenAI capabilities.

For watsonx model:
1. **WATSONX_APIKEY:** This is required for IBM watsonx.governance capabilities. Your Cloud API key can be generated by going to the [**Users** section of the Cloud console](https://cloud.ibm.com/iam#/users). From that page, click your name, scroll down to the **API Keys** section, and click **Create an IBM Cloud API key**. Give your key a name and click **Create**, then copy the created key and paste it below.
2. **WATSONX_PROJECT_ID:** This variable is required for connecting to a watsonx model.
3. **WATSONX_MODEL_ID** This variable specifies the watsonx model ID to be used for requests.
4. **WATSONX_URL** This variable specifies the base URL for the watsonx service endpoint.


Note : In this notebook we are using OPENAI model . Likewise we can also use any model 

In [ ]:
import os

os.environ["OPENAI_API_KEY"]="<EDIT THIS>"

## If you are using watsonx model , set the following properties

# os.environ["WATSONX_API_KEY"]="<EDIT THIS>" 
# os.environ["WATSONX_PROJECT_ID"]="<EDIT THIS>"
# os.environ["WATSONX_MODEL_ID"] = "<EDIT THIS>" # Example : "ibm/granite-3-8b-instruct"
# os.environ["WATSONX_URL"]="<EDIT THIS>" # Example : "https://us-south.ml.cloud.ibm.com"

### Import the needed packages

In [ ]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent
from langchain_mcp_adapters.tools import load_mcp_tools

# Add helper method for display (Can be moved to the package later)
import pandas as pd
from rich.console import Console
from rich.table import Table
def display_dataframe_rich(df: pd.DataFrame, title: str = "DataFrame"):
    console = Console()
    table = Table(title=title, show_lines=True)

    # Add columns
    for column in df.columns:
        table.add_column(str(column))

    # Add rows
    for _, row in df.iterrows():
        table.add_row(*[str(cell) for cell in row])

    console.print(table)

Select the model

In [ ]:
model="gpt-4o-mini" 

#Uncomment the following if you are using ChatWatsonx llm
# from langchain_ibm import ChatWatsonx

# parameters = {
#     "temperature": 0.4
#     }

# model = ChatWatsonx(
#     model_id=os.getenv("WATSONX_MODEL_ID"),
#     apikey=os.getenv("WATSONX_API_KEY"),
#     project_id=os.getenv("WATSONX_PROJECT_ID"),
#     url=os.getenv("WATSONX_URL"),
#     params=parameters
# )

### Intialize the MCP client with transport=stdio

* For **WATSONX_APIKEY:** This is required for IBM watsonx.governance capabilities. Your Cloud API key can be generated by going to the [**Users** section of the Cloud console](https://cloud.ibm.com/iam#/users). From that page, click your name, scroll down to the **API Keys** section, and click **Create an IBM Cloud API key**. Give your key a name and click **Create**, then copy the created key and paste it below.

* For **WATSONX_REGION:** Set if you are using IBM watsonx.governance as a service in a regional data center other than default **Dallas (us-south), in Texas US**. Supported region values are "us-south", "eu-de", "au-syd", "ca-tor", "jp-tok".

In [ ]:
# Set the envioronment variables needed for MCP server or alternatively set the env using .emv.example and load it here 
# from dotenv import dotenv_values
# env = dotenv_values(dotenv_path="/path/to/.env")

env = {
    "WATSONX_APIKEY" : "<EDIT THIS>",
    "WATSONX_REGION": "<EDIT THIS>" # Example "us-south"
}

client = MultiServerMCPClient(
    {
        "governed-catalog": {
            "transport": "stdio",
            "command":"uv",
            "args": ["run", "ibm-watsonx-gov-catalog-mcp-server"],
            "env":env
        },
    })

print("Created the mcp client in stdio transport mode")

Created the mcp client in stdio transport mode


## Find the list of available tools

In [6]:
async def get_mcp_tools(): 
    import json
    tools = []
    try:
        print("Attempting to create session...")
        async with client.session(server_name="governed-catalog") as session:
            try:
                mcp_tools = await load_mcp_tools(session)
            
                available_tools = [{
                    "name": tool.name,
                    "input_schema": tool.args_schema,
                    "description": tool.description,
                } for tool in mcp_tools]
                
                tools_df = pd.DataFrame(available_tools)
                #tools_df['input_schema'] = tools_df['input_schema'].apply(json.dumps)
                display_dataframe_rich(tools_df)
                return available_tools
            except Exception as e:
                    print(f"Failed to load MCP tools: {e}")
                    raise
    except Exception as e:
        print(f"Failed to create session: {e}")
        raise

available_tools = await get_mcp_tools()

Attempting to create session...


                                                     DataFrame                                                     
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ name                        ┃ input_schema                            ┃ description                             ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ add1                        │ {'title': 'add1_schema', 'type':        │ Adds 2 numbers                          │
│                             │ 'object', 'properties': {'num1':        │                                         │
│                             │ {'description': 'First number', 'type': │                                         │
│                             │ 'integer'}, 'num2': {'description':     │                                         │
│                             │ 'Second number', 'type': 'integer'}},   │                                         │
│                             │ 'required': []}                         │                                         │
├─────────────────────────────┼─────────────────────────────────────────┼─────────────────────────────────────────┤
│ jailbreak_detector          │ {'title': 'JailBreakInput', 'type':     │ Tool that detects jailbreak attempts in │
│                             │ 'object', 'properties': {'input':       │ an input prompt.                        │
│                             │ {'description': 'Input text', 'title':  │                                         │
│                             │ 'Input', 'type': 'string'},             │                                         │
│                             │ 'threshold': {'description': 'Threshold │                                         │
│                             │ for calling the tool', 'title':         │                                         │
│                             │ 'Threshold', 'type': 'string'}},        │                                         │
│                             │ 'required': ['input']}                  │                                         │
├─────────────────────────────┼─────────────────────────────────────────┼─────────────────────────────────────────┤
│ answer_relevance_detector   │ {'title':                               │ Tool that detects whether the generated │
│                             │ 'AnswerRelevanceDetectorInput', 'type': │ text meaningfully and directly          │
│                             │ 'object', 'properties': {'input':       │ addresses the user's input.             │
│                             │ {'description': 'Input text', 'title':  │                                         │
│                             │ 'Input', 'type': 'string'},             │                                         │
│                             │ 'generated_text': {'description':       │                                         │
│                             │ 'Response from LLM', 'title':           │                                         │
│                             │ 'Generated Text', 'type': 'string'},    │                                         │
│                             │ 'threshold': {'description': 'Threshold │                                         │
│                             │ for calling the tool', 'title':         │                                         │
│                             │ 'Threshold', 'type': 'string'}},        │                                         │
│                             │ 'required': ['input',                   │                                         │
│                             │ 'generated_text']}                      │                                         │
├─────────────────────────────┼─────────────────────────────────────────┼─────────────────────────────────────────┤
│ topic_relevance_detector    │ {'title':               

## Create a langchain agent using the tools available via governance catalog mcp server

1. Select the tool to be used and create the spec

In [ ]:
#Select the tools
selected_tools = ["wikipedia_search_tool","weather_tool","get_tool","delete_tool"]

#Get the tool spec
selected_tool_spec = []
for tool in available_tools:
    if tool.get("name") in selected_tools:
        selected_tool_spec.append(tool)
        
#Create the agent
ai_gov_agent = create_react_agent(model=model, tools=selected_tool_spec,prompt=(
                "You are AI assistant who will answer the user query using the relevant tool"
                "Always show the output of the function call"
            ))

In [8]:
from langchain_core.messages import (AIMessage, HumanMessage, SystemMessage,
                                     ToolMessage)
async def run_agent(user_input):   
    try:
        print("Attempting to create session...")
        async with client.session(server_name="governed-catalog") as session:
            try:
                messages = [
                        {
                            "role": "user",
                            "content": user_input
                        }
                    ]
            
                response = await ai_gov_agent.ainvoke({"messages": messages})    
            except Exception as e:
                print(f"Error: {str(e)}")
                raise
            
            # Process response and handle tool calls
            tool_results = []
            final_text = []

            # Retain messages
            res_messages = response["messages"]
            #print(res_messages)
            for message in res_messages:
                if isinstance(message, AIMessage):
                    if len(message.content) > 0:
                        final_text.append(message.content)
                        continue
                    
                    tool_calls = message.tool_calls or []
                    for tool_call in tool_calls:
                        tool_name = tool_call.get("name").lower()
                        tool_args = tool_call.get('args')
                        
                        result = await session.call_tool(tool_name, tool_args)
                        tool_results.append({"call": tool_name, "result": result})
                        final_text.append(
                            f"[Calling tool {tool_name} with args {tool_args}]")

                        # Continue conversation with tool results
                        content = result.content[0]
                        if hasattr(content, 'text') and content.text:
                            messages.append({
                                "role": "assistant",
                                "content": content.text
                            })

                            final_text.append(result.content[0].text)
                else:
                    final_text.append(message.content)

        return "\n".join(final_text)
    except Exception as e:
            print(f"Failed to create session: {e}")
            raise

### Run agent with mcp tools

In [9]:
# result = await run_agent("How is the weather in hyderabad?")
# result
result = await run_agent("What is prompt injection")
print(result)

Attempting to create session...
What is prompt injection
[Calling tool wikipedia_search_tool with args {'query': 'prompt injection'}]
Page: Prompt injection
Summary: Prompt injection is a cybersecurity exploit in which adversaries craft inputs that appear legitimate but are designed to cause unintended behavior in  machine learning models, particularly large language models (LLMs). This attack takes advantage of the model's inability to distinguish between developer-defined prompts and user inputs, allowing adversaries to bypass safeguards and influence model behaviour. While LLMs are designed to follow trusted instructions, they can be manipulated into carrying out unintended responses through carefully crafted inputs.
With capabilities such as web browsing and file upload, an LLM not only needs to differentiate from developer instructions from user input, but also to differentiate user input from content not directly authored by the user. LLMs with web browsing capabilities can be ta

In [ ]:
result = await run_agent("Get the details of weather_tool")
print(result)

Attempting to create session...
Get the details of weather_tool
[Calling tool get_tool with args {'tool_name': 'weather_tool'}]
{'entity': {'display_name': 'Weather', 'tool_name': 'weather_tool', 'component_type': 'ai_tool', 'service_provider_type': 'IBM', 'tool_type': 'code', 'description': 'Retrieve the weather for a given location', 'is_draft': False, 'summary': '', 'development_uri_link': '', 'inventory_id': '58f75478-cb36-4fb3-a32e-fa84b6671e9a', 'reusable': True, 'category': ['Other'], 'used_in_applications': [], 'code': {'source': None, 'commit': '', 'language': 'python', 'source_code_url': None, 'source_code_base64': '', 'run_time_details': {'engine': 'Python 3.11', 'cpu_capacity': '', 'memory': ''}}, 'framework': ['langgraph', 'langchain'], 'metrics': {}, 'benchmark_test': {'avg_latency': '', 'dataset': '', 'records': 0}, 'pricing': {}, 'endpoint': {'url': None, 'headers': {}, 'method': 'POST'}, 'dependencies': {'remote_services': [], 'run_time_packages': []}, 'config': {}, 's

Congratulations! You are now able to use the MCP server that helps in using/registering/getting the details of tools hosted on Governed Catalog in your agent . 

Author: [Sowmya Kollipara](sokollip@in.ibm.com)